In [85]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

np.set_printoptions(suppress=True, precision=3)

sns.set(style='ticks', palette='Set2')
sns.despine()

In [86]:
import utils as U
import opt_runs as OR
import opt_model as OM
import decorated_options as Deco

The RedQueen demo expects data in the following format:

```
{
  "walls": {
    "1": [1,2,3,4,5], # These time-stamps do not contain posts by our user.
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": [1, 4],
    "poisson": [1, 3]
  }
}
```

However, this shifts the task of calculating performance to the JS library, which will require re-implementation and potential bugs. Hence, the performance data should be included with this data.

```
{
  "walls": {
    "1": [1,2,3,4,5],
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": {
        "post_times": [1, 4],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
     }, 
    "poisson": {
        "post_times": [1, 3],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
    },
  }
}
```

The `<metric>` could be `perf_top_{1,5,10}`, `avg_rank`, `r_2_int`, etc.

In [3]:
# Step 1: Create a SimOpts which will generate raw data

In [71]:
sim_opts_1_follower = OM.SimOpts(
    src_id=0,
    end_time=100,
    q_vec=np.array([1]),
    s=1.0,
    other_sources=[('Hawkes', {'src_id': 1, 'seed': 1, 'l_0': 1.0, 'alpha': 1.0, 'beta': 5.0})],
    sink_ids=[1000],
    edge_list=[(0, 1000), (1, 1000)]
)

In [1]:
# Step 2: Simulate the walls along with different strategies for our broadcaster

In [72]:
%%time
seed = 1
opt_mgr = sim_opts_1_follower.create_manager_with_opt(seed)
opt_mgr.run_dynamic()
opt_df = opt_mgr.state.get_dataframe()
num_opt_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_opt = {
    'type': 'Opt',
    'seed': seed,
    'capacity': num_opt_tweets,
    's': sim_opts_1_follower.s
}
OR.add_perf(perf_opt, opt_df, sim_opts_1_follower)

CPU times: user 117 ms, sys: 3.33 ms, total: 120 ms
Wall time: 115 ms


In [73]:
%%time
seed = 9
poisson_mgr = sim_opts_1_follower.create_manager_with_poisson(seed, capacity=num_opt_tweets)
poisson_mgr.run_dynamic()
poisson_df = poisson_mgr.state.get_dataframe()
num_poisson_tweets = U.num_tweets_of(opt_df, broadcaster_id=0)
perf_poisson = {
    'type': 'Poisson',
    'seed': seed,
    'capacity': num_poisson_tweets,
    's': sim_opts_1_follower.s
}
OR.add_perf(perf_poisson, poisson_df, sim_opts_1_follower)

CPU times: user 96.7 ms, sys: 3.33 ms, total: 100 ms
Wall time: 98.4 ms


In [25]:
# Step 3: Calculate the performance and save JSON output

In [74]:
print('num_opt_tweets = {}, num_poisson_tweets = {}'
      .format(U.num_tweets_of(opt_df, 0), U.num_tweets_of(poisson_df, 0)))

num_opt_tweets = 78.0, num_poisson_tweets = 78.0


In [75]:
print('avg_rank_opt = {}, avg_rank_poisson = {}'
      .format(U.average_rank(opt_df, sim_opts=sim_opts_1_follower), 
              U.average_rank(poisson_df, sim_opts=sim_opts_1_follower)))

avg_rank_opt = 66.8116376874034, avg_rank_poisson = 201.3461740580856


In [76]:
perf_opt

{'avg_rank': 66.811637687403405,
 'capacity': 78.0,
 'num_events': 78,
 'r_2': 138.15727122691501,
 's': 1.0,
 'seed': 1,
 'top_1': 56.517519051405124,
 'type': 'Opt',
 'world_events': 154}

In [77]:
print('top_1_opt = {}, top_1_poisson = {}'
      .format(perf_opt['top_1'], perf_poisson['top_1']))

top_1_opt = 56.517519051405124, top_1_poisson = 39.68728199394901


The output JSON structure, for reference:

```
{
  "walls": {
    "1": [1,2,3,4,5],
    "2": [1,4,5]
  },
  "broadcasts": {
    "redqueen": {
        "post_times": [1, 4],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
     }, 
    "poisson": {
        "post_times": [1, 3],
        "performance": {
            "<metric>": [(<timestamp>, <perf>)]
        }
    },
  }
}
```

In [78]:
perf_opt

{'avg_rank': 66.811637687403405,
 'capacity': 78.0,
 'num_events': 78,
 'r_2': 138.15727122691501,
 's': 1.0,
 'seed': 1,
 'top_1': 56.517519051405124,
 'type': 'Opt',
 'world_events': 154}

In [79]:
tmp = U.rank_of_src_in_df(opt_df, 0).mean(1)
list(zip(tmp.index, tmp.values))

[(0.0, 0.0),
 (0.53960583725918543, 1.0),
 (0.53972021861782848, 2.0),
 (0.61909770798845698, 3.0),
 (0.61921208934710004, 0.0),
 (0.70701425826167574, 1.0),
 (0.90834712903543691, 2.0),
 (1.0670270131155948, 0.0),
 (1.1917144404267215, 1.0),
 (1.2885983120804549, 0.0),
 (1.3478466857159106, 1.0),
 (1.3644643223058284, 2.0),
 (1.5539613197979911, 0.0),
 (1.5772781255977351, 1.0),
 (1.6579275545759009, 2.0),
 (1.8248160139485703, 3.0),
 (1.8697462900750719, 4.0),
 (1.9418837295870937, 5.0),
 (1.9788719733383855, 6.0),
 (2.0827306673084838, 0.0),
 (2.3327966872249237, 1.0),
 (2.3445771877269492, 2.0),
 (2.3605663119933471, 0.0),
 (2.47743895623874, 1.0),
 (2.5253334173526443, 2.0),
 (3.0175300026831691, 0.0),
 (3.3827283444815013, 1.0),
 (3.5340012640634368, 0.0),
 (3.709553894911068, 1.0),
 (3.7258160278525145, 2.0),
 (3.8385804132380663, 3.0),
 (3.9303271190277984, 0.0),
 (4.1460630671717382, 1.0),
 (4.5221016800807083, 0.0),
 (4.725618040142133, 1.0),
 (5.5486743603649726, 2.0),
 (5.6

In [126]:
@Deco.optioned('opts')
def perf_to_json(dfs, names, src_id, sink_ids, end_time):
    """Produce a dictionary which captures performance for the demo app."""        
    
    # Assumes that the walls across the data frames are the same.
    eg_df = dfs[0]
    
    walls = {}
    for sink_id in sink_ids:
        walls[sink_id] = eg_df[(eg_df.src_id != src_id) & (eg_df.sink_id == sink_id)].t.tolist()
    
    broadcasts = {}
    for df, name in zip(dfs, names):
        r_t = U.rank_of_src_in_df(df, src_id)
        avg_rank = r_t.mean(1)
        time_at_top = np.where(r_t < 1.0, 1.0, 0.0).mean(1)
        dt = np.diff(np.concatenate([r_t.index.values, [end_time]]))
        broadcasts[name] = {
            'post_times': df[df.src_id == src_id].t.unique().tolist(),
            'performance': {
                'avg_rank': list(zip(avg_rank.index, avg_rank.values)),
                'time_at_top': list(zip(avg_rank.index, np.cumsum(time_at_top * dt)))
            }
        }
        
    return {
        'walls': walls,
        'broadcasts': broadcasts
    }


In [127]:
example_1 = perf_to_json([opt_df, poisson_df], ['redqueen', 'poisson'], 
                         opts=Deco.Options(**sim_opts_1_follower.get_dict()))

In [117]:
U.time_in_top_k(opt_df, src_id=1, K=1, sim_opts=sim_opts_1_follower)

43.482480948594869

In [123]:
example_1['broadcasts']

{'poisson': {'performance': {'avg_rank': [(0.53960583725918543, 1.0),
    (0.53972021861782848, 2.0),
    (0.61909770798845698, 3.0),
    (0.70701425826167574, 4.0),
    (0.90834712903543691, 5.0),
    (1.1917144404267215, 6.0),
    (1.3478466857159106, 7.0),
    (1.3644643223058284, 8.0),
    (1.5772781255977351, 9.0),
    (1.6579275545759009, 10.0),
    (1.8248160139485703, 11.0),
    (1.8697462900750719, 12.0),
    (1.9418837295870937, 13.0),
    (1.9788719733383855, 14.0),
    (2.2866949969676864, 0.0),
    (2.3327966872249237, 1.0),
    (2.3445771877269492, 2.0),
    (2.47743895623874, 3.0),
    (2.5253334173526443, 4.0),
    (3.3827283444815013, 5.0),
    (3.709553894911068, 6.0),
    (3.7258160278525145, 7.0),
    (3.8385804132380663, 8.0),
    (3.8570923162358328, 0.0),
    (3.8748375832319826, 0.0),
    (3.9454817674267062, 0.0),
    (4.1460630671717382, 1.0),
    (4.725618040142133, 2.0),
    (5.3059424485393025, 0.0),
    (5.5486743603649726, 1.0),
    (5.6547599651701477, 2

In [124]:
import json

In [128]:
with open('data/example2.json', 'w') as f:
    json.dump(example_1, f, indent=2)

## Scratch pad

In [107]:


world_seed=42; world_rate=1000.0
sim_opts = OM.SimOpts(src_id=1,
                       other_sources=[('Poisson2',
                                       {'src_id': 2,
                                        'seed': world_seed,
                                        'rate': world_rate}),
                                     ('Poisson2',
                                       {'src_id': 3,
                                        'seed': world_seed + 2,
                                        'rate': 2*world_rate}),
                                     ('Poisson2',
                                       {'src_id': 4,
                                        'seed': world_seed + 2,
                                        'rate': world_rate})],
                       end_time=1.0,
                       sink_ids=[1001],
                       q_vec=np.asarray([1.0]),
                       s=1.0,
                       edge_list=[(2, 1001), (3, 1001)])


In [108]:
mgr = sim_opts.create_manager_for_wall()

In [109]:
%%time
mgr.run_dynamic()

CPU times: user 50 ms, sys: 3.33 ms, total: 53.3 ms
Wall time: 51.2 ms


In [110]:
df = mgr.get_state().get_dataframe()

In [111]:
df.shape

(3037, 5)

In [112]:
U.time_in_top_k(df, src_id=2, K=1, end_time=sim_opts.end_time)

0.33151399927021774

In [98]:
df

,event_id,sink_id,src_id,t,time_delta
0,100,1001,3,0.003218,3.218264e-03
1,101,1001,2,0.004632,1.413759e-03
2,102,1001,3,0.004632,0.000000e+00
3,104,1001,3,0.004940,3.079579e-04
4,105,1001,2,0.005062,1.216029e-04
5,106,1001,3,0.005062,0.000000e+00
6,108,1001,3,0.005230,1.680297e-04
7,109,1001,2,0.005522,2.925036e-04
8,110,1001,3,0.005522,0.000000e+00
9,112,1001,3,0.005759,2.365434e-04
